<a href="https://www.kaggle.com/code/dwijmistry/sign-language-challenge-google?scriptVersionId=121288244" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Google - Isolated Sign Language Recognition

The goal of this competition is to classify isolated American Sign Language (ASL) signs.

The landmarks were extracted from raw videos with the MediaPipe holistic model and are asked to predict the sign from this data.

## Version 1
- Based on https://www.kaggle.com/code/robikscube/sign-language-recognition-eda-twitch-stream

## Version 2
- Added remaining code

## Version 3
- Based on https://www.kaggle.com/code/medali1992/gislr-nn-arcface-baseline/notebook (Version 26)

# pip Install

In [1]:
!pip install onnx_tf
!pip install tflite-runtime
!pip install -q --upgrade wandb

# install nb_black for sutoformatting
!pip install nb_black --quiet
%load_ext lab_black

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.9 MB/s eta 0:00:00


# Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("seaborn-colorblind")


import math
import random
import time
from collections import OrderedDict
import tensorflow as tf
from tqdm import tqdm
import json
import os
import gc
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GroupKFold, StratifiedGroupKFold

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.optim.optimizer import Optimizer
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import (
    CosineAnnealingWarmRestarts,
    CosineAnnealingLR,
    ReduceLROnPlateau,
)
from torchinfo import summary

import onnx
import onnx_tf
from onnx_tf.backend import prepare

import warnings

warnings.filterwarnings("ignore")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
VERSION = 14
DATA_DIR = "/kaggle/input/asl-signs"

# Utils

In [3]:
# ====================================================
# Utils
# ====================================================
def get_score(y_true, y_pred):
    score = accuracy_score(y_true, y_pred)
    return score


def seed_torch(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


def load_relevant_data_subset_with_imputation(pq_path):
    data_columns = ["x", "y", "z"]
    data = pd.read_parquet(pq_path, columns=data_columns)
    data.replace(np.nan, 0, inplace=True)
    n_frames = int(len(data) / CFG.rows_per_frame)
    data = data.values.reshape(n_frames, CFG.rows_per_frame, len(data_columns))
    return data.astype(np.float32)


def load_relevant_data_subset(pq_path):
    data_columns = ["x", "y"]
    data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / CFG.rows_per_frame)
    data = data.values.reshape(n_frames, CFG.rows_per_frame, len(data_columns))
    return data.astype(np.float32)


def read_dict(file_path):
    path = os.path.expanduser(file_path)
    with open(path, "r") as f:
        dic = json.load(f)
    return dic

# Configuration

In [4]:
class CFG:
    num_workers = 2
    apex = False
    scheduler = "CosineAnnealingLR"  # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts','OneCycleLR']
    epochs = 500
    print_freq = 200
    # CosineAnnealingLR params
    cosanneal_params = {"T_max": 5, "eta_min": 3 * 1e-5, "last_epoch": -1}
    # ReduceLROnPlateau params
    reduce_params = {
        "mode": "max",
        "factor": 0.8,
        "patience": 5,
        "eps": 1e-6,
        "verbose": True,
    }
    # CosineAnnealingWarmRestarts params
    cosanneal_res_params = {"T_0": 3, "eta_min": 1e-6, "T_mult": 1, "last_epoch": -1}
    # OneCycleLR params
    onecycle_params = {
        "pct_start": 0.1,
        "div_factor": 1e1,
        "max_lr": 1e-3,
        "steps_per_epoch": 3,
        "epochs": 3,
    }
    momentum = 0.9
    model_name = "NN_ArcFace"
    lr = 0.000333
    weight_decay = 1e-4
    gradient_accumulation_steps = 1
    max_grad_norm = 1000
    data_path = "../input/asl-signs/"
    debug = False
    arcface = False
    use_aggregation_dataset = True
    target_size = 250
    rows_per_frame = 543
    batch_size = 512
    train = True
    early_stop = True
    target_col = "label"
    scale = 30.0
    margin = 0.50
    easy_margin = False
    ls_eps = 0.0
    fc_dim = 512
    early_stopping_steps = 5
    grad_cam = False
    seed = 42

# Directory Settings


In [5]:
# ====================================================
# Directory settings
# ====================================================
import os

OUTPUT_DIR = f"./{CFG.model_name}_version{VERSION}/"
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)


def init_logger(log_file=OUTPUT_DIR + "train.log"):
    from logging import getLogger, INFO, FileHandler, Formatter, StreamHandler

    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger


LOGGER = init_logger()

# Load Data


In [6]:
train = pd.read_csv(f"{CFG.data_path}train.csv")
label_index = read_dict(f"{CFG.data_path}sign_to_prediction_index_map.json")
index_label = dict([(label_index[key], key) for key in label_index])
train["label"] = train["sign"].map(lambda sign: label_index[sign])
train["path"] = DATA_DIR + "/" + train["path"]

if CFG.debug:
    CFG.epochs = 1
    train = train.sample(n=4000, random_state=CFG.seed).reset_index(drop=True)

train.head()

,path,participant_id,sequence_id,sign,label
0,/kaggle/input/asl-signs/train_landmark_files/2...,26734,1000035562,blow,25
1,/kaggle/input/asl-signs/train_landmark_files/2...,28656,1000106739,wait,232
2,/kaggle/input/asl-signs/train_landmark_files/1...,16069,100015657,cloud,48
3,/kaggle/input/asl-signs/train_landmark_files/2...,25571,1000210073,bird,23
4,/kaggle/input/asl-signs/train_landmark_files/6...,62590,1000240708,owie,164


# Data Preparation¶

In [7]:
DROP_Z = True

NUM_FRAMES = 15
SEGMENTS = 3

LEFT_HAND_OFFSET = 468
POSE_OFFSET = LEFT_HAND_OFFSET + 21
RIGHT_HAND_OFFSET = POSE_OFFSET + 33

## average over the entire face, and the entire 'pose'
averaging_sets = [[0, 468], [POSE_OFFSET, 33]]

lip_landmarks = [
    61,
    185,
    40,
    39,
    37,
    0,
    267,
    269,
    270,
    409,
    291,
    146,
    91,
    181,
    84,
    17,
    314,
    405,
    321,
    375,
    78,
    191,
    80,
    81,
    82,
    13,
    312,
    311,
    310,
    415,
    95,
    88,
    178,
    87,
    14,
    317,
    402,
    318,
    324,
    308,
]
left_hand_landmarks = list(range(LEFT_HAND_OFFSET, LEFT_HAND_OFFSET + 21))
right_hand_landmarks = list(range(RIGHT_HAND_OFFSET, RIGHT_HAND_OFFSET + 21))

point_landmarks = [
    item
    for sublist in [lip_landmarks, left_hand_landmarks, right_hand_landmarks]
    for item in sublist
]

LANDMARKS = len(point_landmarks) + len(averaging_sets)
print(LANDMARKS)
if DROP_Z:
    INPUT_SHAPE = (NUM_FRAMES, LANDMARKS * 2)
else:
    INPUT_SHAPE = (NUM_FRAMES, LANDMARKS * 3)

FLAT_INPUT_SHAPE = (INPUT_SHAPE[0] + 2 * (SEGMENTS + 1)) * INPUT_SHAPE[1]

84


In [8]:
def tf_nan_mean(x, axis=0):
    return tf.reduce_sum(
        tf.where(tf.math.is_nan(x), tf.zeros_like(x), x), axis=axis
    ) / tf.reduce_sum(
        tf.where(tf.math.is_nan(x), tf.zeros_like(x), tf.ones_like(x)), axis=axis
    )


def tf_nan_std(x, axis=0):
    d = x - tf_nan_mean(x, axis=axis)
    return tf.math.sqrt(tf_nan_mean(d * d, axis=axis))


def flatten_means_and_stds(x, axis=0):
    # Get means and stds
    x_mean = tf_nan_mean(x, axis=0)
    x_std = tf_nan_std(x, axis=0)

    x_out = tf.concat([x_mean, x_std], axis=0)
    x_out = tf.reshape(x_out, (1, INPUT_SHAPE[1] * 2))
    x_out = tf.where(tf.math.is_finite(x_out), x_out, tf.zeros_like(x_out))
    return x_out

In [9]:
class FeatureGen(tf.keras.layers.Layer):
    def __init__(self):
        super(FeatureGen, self).__init__()

    def call(self, x_in):
        #         print(right_hand_percentage(x))
        x_list = [
            tf.expand_dims(
                tf_nan_mean(x_in[:, av_set[0] : av_set[0] + av_set[1], :], axis=1),
                axis=1,
            )
            for av_set in averaging_sets
        ]
        x_list.append(tf.gather(x_in, point_landmarks, axis=1))
        x = tf.concat(x_list, 1)

        x_padded = x
        for i in range(SEGMENTS):
            p0 = tf.where(
                ((tf.shape(x_padded)[0] % SEGMENTS) > 0) & ((i % 2) != 0), 1, 0
            )
            p1 = tf.where(
                ((tf.shape(x_padded)[0] % SEGMENTS) > 0) & ((i % 2) == 0), 1, 0
            )
            paddings = [[p0, p1], [0, 0], [0, 0]]
            x_padded = tf.pad(x_padded, paddings, mode="SYMMETRIC")
        x_list = tf.split(x_padded, SEGMENTS)
        x_list = [flatten_means_and_stds(_x, axis=0) for _x in x_list]

        x_list.append(flatten_means_and_stds(x, axis=0))

        ## Resize only dimension 0. Resize can't handle nan, so replace nan with that dimension's avg value to reduce impact.
        x = tf.image.resize(
            tf.where(tf.math.is_finite(x), x, tf_nan_mean(x, axis=0)),
            [NUM_FRAMES, LANDMARKS],
        )
        x = tf.reshape(x, (1, INPUT_SHAPE[0] * INPUT_SHAPE[1]))
        x = tf.where(tf.math.is_nan(x), tf.zeros_like(x), x)
        x_list.append(x)
        x = tf.concat(x_list, axis=1)
        return x


feature_converter = FeatureGen()

X = np.load(
    "/kaggle/input/gislr-feature-data-on-the-shoulders/feature_data.npy"
).astype(np.float32)
y = np.load(
    "/kaggle/input/gislr-feature-data-on-the-shoulders/feature_labels.npy"
).astype(np.uint8)
print(X.shape, y.shape)

if DROP_Z:
    X = np.reshape(X, [X.shape[0], -1, 3])
    X = X[:, :, 0:2]
    X = np.reshape(X, [X.shape[0], -1])
    print(X.shape, y.shape)

(94477, 5796) (94477,)
(94477, 3864) (94477,)


# Model Tracking

In [10]:
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
wandb_api = user_secrets.get_secret("wandb_key")

import wandb

wandb.login(key=wandb_api)


def class2dict(f):
    return dict(
        (name, getattr(f, name)) for name in dir(f) if not name.startswith("__")
    )


run = wandb.init(
    project="GISLR Competition",
    name=f"{CFG.model_name}_Version{VERSION}",
    config=class2dict(CFG),
    group=CFG.model_name,
    job_type="train",
)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dwijmistry. Use `wandb login --relogin` to force relogin


# DataSet

In [11]:
class Dataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, i):
        return self.X[i].astype(np.float32), self.y[i]

# Model

In [12]:
class ArcMarginProduct(nn.Module):
    def __init__(
        self,
        in_features,
        out_features,
        scale=30.0,
        margin=0.50,
        easy_margin=False,
        ls_eps=0.0,
    ):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.scale = scale
        self.margin = margin
        self.ls_eps = ls_eps  # label smoothing
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(margin)
        self.sin_m = math.sin(margin)
        self.th = math.cos(math.pi - margin)
        self.mm = math.sin(math.pi - margin) * margin

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size(), device=device)
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.out_features
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.scale

        return output


# https://github.com/haqishen/Google-Landmark-Recognition-2020-3rd-Place-Solution/blob/main/landmark-recognition-2020-third-place-submission.ipynb
class ArcMarginProduct_subcenter(nn.Module):
    def __init__(self, in_features, out_features, k=3):
        super().__init__()
        self.weight = nn.Parameter(torch.FloatTensor(out_features * k, in_features))
        self.reset_parameters()
        self.k = k
        self.out_features = out_features

    def reset_parameters(self):
        stdv = 1.0 / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)

    def forward(self, features):
        cosine_all = F.linear(F.normalize(features), F.normalize(self.weight))
        cosine_all = cosine_all.view(-1, self.out_features, self.k)
        cosine, _ = torch.max(cosine_all, dim=2)
        return cosine

In [13]:
class ASLLinearModel(torch.nn.Module):
    def __init__(
        self,
        in_features: int,
        first_out_features: int,
        num_classes: int,
        num_blocks: int,
        drop_rate: float,
    ):
        super(ASLLinearModel, self).__init__()

        blocks = []
        out_features = first_out_features
        for idx in range(num_blocks):
            blocks.append(self._make_block(in_features, out_features, drop_rate))

            in_features = out_features
            out_features = out_features // 2

        self.model = nn.Sequential(*blocks)
        self.final = ArcMarginProduct(
            256,
            num_classes,
            scale=CFG.scale,
            margin=CFG.margin,
            easy_margin=False,
            ls_eps=0.0,
        )
        self.fc_probs = nn.Linear(256, num_classes)
        print(self.model)

    def _make_block(self, in_features, out_features, drop_rate):
        return nn.Sequential(
            nn.Linear(in_features, out_features),
            nn.BatchNorm1d(out_features),
            nn.ReLU(),
            nn.Dropout(drop_rate),
        )

    def forward(self, x, label):
        feature = self.model(x)
        if CFG.arcface:
            arcface = self.final(feature, label)
            probs = self.fc_probs(feature)
            return probs, arcface
        else:
            probs = self.fc_probs(feature)
            return probs

# Helper Function

In [14]:
# ====================================================
# Helper functions
# ====================================================
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return "%dm %ds" % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return "%s (remain %s)" % (asMinutes(s), asMinutes(rs))


def train_fn(train_loader, model, criterion, optimizer, epoch, scheduler, device):
    if CFG.apex:
        scaler = GradScaler()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    # switch to train mode
    model.train()
    start = end = time.time()
    global_step = 0
    for step, (features, labels) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        features = features.to(device).float()
        labels = labels.to(device).long()
        batch_size = labels.size(0)
        if CFG.apex:
            with autocast():
                if CFG.arcface:
                    probs, arcface = model(features, labels)
                    arcface_loss = nn.CrossEntropyLoss()(arcface, labels)
                    loss = criterion(probs, labels)
                else:
                    y_preds = model(features, labels)
                    loss = criterion(y_preds, labels)
        else:
            if CFG.arcface:
                probs, arcface = model(features, labels)
                arcface_loss = nn.CrossEntropyLoss()(arcface, labels)
                loss = criterion(probs, labels)
            else:
                y_preds = model(features, labels)
                loss = criterion(y_preds, labels)
        # record loss
        if CFG.arcface:
            loss = 0.5 * loss + 0.5 * arcface_loss
            losses.update(loss.item(), batch_size)
        else:
            losses.update(loss.item(), batch_size)

        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        if CFG.apex:
            scaler.scale(loss).backward()
        else:
            loss.backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(
            model.parameters(), CFG.max_grad_norm
        )
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            if CFG.apex:
                scaler.step(optimizer)
                scaler.update()
            else:
                optimizer.step()
            optimizer.zero_grad()
            global_step += 1
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader) - 1):
            print(
                "Epoch: [{0}][{1}/{2}] "
                "Elapsed {remain:s} "
                "Loss: {loss.val:.4f}({loss.avg:.4f}) "
                "Grad: {grad_norm:.4f} "
                "LR: {lr:.6f}  ".format(
                    epoch + 1,
                    step,
                    len(train_loader),
                    remain=timeSince(start, float(step + 1) / len(train_loader)),
                    loss=losses,
                    grad_norm=grad_norm,
                    lr=scheduler.get_lr()[0],
                )
            )
        wandb.log(
            {
                f"loss": losses.val,
                f"lr": scheduler.get_lr()[0],
            }
        )
    return losses.avg


def valid_fn(valid_loader, model, criterion, device):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    # switch to evaluation mode
    model.eval()
    preds = []
    start = end = time.time()
    for step, (features, labels) in enumerate(valid_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        features = features.to(device).float()
        labels = labels.to(device).long()
        batch_size = labels.size(0)
        # compute loss
        with torch.no_grad():
            if CFG.arcface:
                y_preds, _ = model(features, labels)
            else:
                y_preds = model(features, labels)

        preds.append(y_preds.softmax(1).to("cpu").numpy())
        loss = criterion(y_preds, labels)
        losses.update(loss.item(), batch_size)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader) - 1):
            print(
                "EVAL: [{0}/{1}] "
                "Elapsed {remain:s} "
                "Loss: {loss.val:.4f}({loss.avg:.4f}) ".format(
                    step,
                    len(valid_loader),
                    loss=losses,
                    remain=timeSince(start, float(step + 1) / len(valid_loader)),
                )
            )
    predictions = np.concatenate(preds)
    return losses.avg, predictions

# Train Loop¶


In [15]:
# Seed for producing results
seed_torch(seed=45)

# ====================================================
# loader
# ====================================================
groups = train["path"].map(lambda x: x.split("/")[5])
sgkf = StratifiedGroupKFold(n_splits=5, random_state=42, shuffle=True)
for i, (train_index, valid_index) in enumerate(sgkf.split(X, y, groups)):
    train_index = train_index
    valid_index = valid_index
    print(f"Fold {i}:")
    print(f"  Train index shape: {train_index.shape}")
    print(f"         group={groups[train_index]}")
    print(f"  Valid index shape:  {valid_index.shape}")
    print(f"         group={groups[valid_index]}")
    break
X_train, X_val, y_train, y_val = (
    X[train_index],
    X[valid_index],
    y[train_index],
    y[valid_index],
)
train_dataset = Dataset(X_train, y_train)
valid_dataset = Dataset(X_val, y_val)


train_loader = DataLoader(
    train_dataset,
    batch_size=CFG.batch_size,
    shuffle=True,
    num_workers=CFG.num_workers,
    pin_memory=True,
    drop_last=True,
)
valid_loader = DataLoader(
    valid_dataset,
    batch_size=CFG.batch_size,
    shuffle=False,
    num_workers=CFG.num_workers,
    pin_memory=True,
    drop_last=False,
)


# ====================================================
# scheduler
# ====================================================
def get_scheduler(optimizer):
    if CFG.scheduler == "ReduceLROnPlateau":
        scheduler = ReduceLROnPlateau(optimizer, **CFG.reduce_params)
    elif CFG.scheduler == "CosineAnnealingLR":
        scheduler = CosineAnnealingLR(optimizer, **CFG.cosanneal_params)
    elif CFG.scheduler == "CosineAnnealingWarmRestarts":
        scheduler = CosineAnnealingWarmRestarts(optimizer, **CFG.reduce_params)
    return scheduler


# ====================================================
# model & optimizer
# ====================================================
model = ASLLinearModel(
    in_features=3864,
    first_out_features=1024,
    num_classes=250,
    num_blocks=3,
    drop_rate=0.4,
)
model.to(device)

optimizer = Adam(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay)
scheduler = get_scheduler(optimizer)

# ====================================================
# loop
# ====================================================
criterion = nn.CrossEntropyLoss()
best_score = 0
for epoch in range(CFG.epochs):
    start_time = time.time()

    # train
    avg_loss = train_fn(
        train_loader, model, criterion, optimizer, epoch, scheduler, device
    )

    # eval
    avg_val_loss, preds = valid_fn(valid_loader, model, criterion, device)

    if isinstance(scheduler, ReduceLROnPlateau):
        scheduler.step(avg_val_loss)
    elif isinstance(scheduler, CosineAnnealingLR):
        scheduler.step()
    elif isinstance(scheduler, CosineAnnealingWarmRestarts):
        scheduler.step()

    score = get_score(y_val, preds.argmax(1))

    elapsed = time.time() - start_time

    LOGGER.info(
        f"Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s"
    )
    LOGGER.info(f"Epoch {epoch+1} - Score: {score:.4f}")
    wandb.log(
        {
            f"epoch": epoch + 1,
            f"avg_train_loss": avg_loss,
            f"avg_val_loss": avg_val_loss,
            f"score": score,
        }
    )

    if best_score < score:
        best_score = score
        LOGGER.info(f"Epoch {epoch+1} - Save Best score: {best_score:.4f} Model")
        torch.save(
            model.state_dict(),
            OUTPUT_DIR + f"{CFG.model_name}_best_score_version{VERSION}.pth",
        )
LOGGER.info(f"Our CV score is {best_score}")

Fold 0:
  Train index shape: (70951,)
         group=0        26734
3        25571
5        26734
6        26734
7        32319
         ...  
94470    61333
94471    25571
94473    26734
94474    25571
94475    29302
Name: path, Length: 70951, dtype: object
  Valid index shape:  (23526,)
         group=1        28656
2        16069
4        62590
11       36257
15       28656
         ...  
94461    62590
94462    62590
94469    53618
94472    53618
94476    36257
Name: path, Length: 23526, dtype: object
Sequential(
  (0): Sequential(
    (0): Linear(in_features=3864, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
  )
  (1): Sequential(
    (0): Linear(in_features=1024, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
  )
  (2): Sequen

Epoch 1 - avg_train_loss: 5.4442  avg_val_loss: 5.1672  time: 6s
Epoch 1 - Score: 0.0174
Epoch 1 - Save Best score: 0.0174 Model


EVAL: [45/46] Elapsed 0m 0s (remain 0m 0s) Loss: 5.1634(5.1672) 
Epoch: [2][0/138] Elapsed 0m 0s (remain 0m 21s) Loss: 5.2075(5.2075) Grad: 1.1028 LR: 0.000278  
Epoch: [2][137/138] Elapsed 0m 2s (remain 0m 0s) Loss: 4.8111(4.9983) Grad: 1.0895 LR: 0.000278  
EVAL: [0/46] Elapsed 0m 0s (remain 0m 5s) Loss: 4.7577(4.7577) 


Epoch 2 - avg_train_loss: 4.9983  avg_val_loss: 4.7822  time: 4s
Epoch 2 - Score: 0.0463
Epoch 2 - Save Best score: 0.0463 Model


EVAL: [45/46] Elapsed 0m 0s (remain 0m 0s) Loss: 4.7853(4.7822) 
Epoch: [3][0/138] Elapsed 0m 0s (remain 0m 22s) Loss: 4.8251(4.8251) Grad: 1.1387 LR: 0.000174  
Epoch: [3][137/138] Elapsed 0m 2s (remain 0m 0s) Loss: 4.5219(4.6059) Grad: 1.1337 LR: 0.000174  
EVAL: [0/46] Elapsed 0m 0s (remain 0m 5s) Loss: 4.5905(4.5905) 


...

EVAL: [45/46] Elapsed 0m 0s (remain 0m 0s) Loss: 2.0023(1.9428) 
Epoch: [499][0/138] Elapsed 0m 0s (remain 0m 20s) Loss: 1.2542(1.2542) Grad: 2.6839 LR: 0.000406  
Epoch: [499][137/138] Elapsed 0m 2s (remain 0m 0s) Loss: 1.3347(1.2862) Grad: 2.8258 LR: 0.000406  
EVAL: [0/46] Elapsed 0m 0s (remain 0m 5s) Loss: 2.1573(2.1573) 


Epoch 499 - avg_train_loss: 1.2862  avg_val_loss: 2.2256  time: 3s
Epoch 499 - Score: 0.4750


EVAL: [45/46] Elapsed 0m 0s (remain 0m 0s) Loss: 2.2692(2.2256) 
Epoch: [500][0/138] Elapsed 0m 0s (remain 0m 21s) Loss: 1.1709(1.1709) Grad: 2.4774 LR: 0.000409  
Epoch: [500][137/138] Elapsed 0m 2s (remain 0m 0s) Loss: 1.4309(1.3794) Grad: 2.9606 LR: 0.000409  
EVAL: [0/46] Elapsed 0m 0s (remain 0m 6s) Loss: 3.1149(3.1149) 


Epoch 500 - avg_train_loss: 1.3794  avg_val_loss: 3.1407  time: 3s
Epoch 500 - Score: 0.3171
Our CV score is 0.574130748958599


EVAL: [45/46] Elapsed 0m 0s (remain 0m 0s) Loss: 3.1870(3.1407) 


# Tensorflow Conversion¶


In [16]:
class Model_infe(ASLLinearModel):
    def __init__(self):
        super().__init__(
            in_features=3864,
            first_out_features=1024,
            num_classes=250,
            num_blocks=3,
            drop_rate=0.4,
        )

    def forward(self, x):
        feature = self.model(x)
        probs = self.fc_probs(feature)
        return probs


model_infe = Model_infe()
model_infe.load_state_dict(
    torch.load(OUTPUT_DIR + f"{CFG.model_name}_best_score_version{VERSION}.pth"),
    strict=False,
)
model_infe = model_infe.to(device)

Sequential(
  (0): Sequential(
    (0): Linear(in_features=3864, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
  )
  (1): Sequential(
    (0): Linear(in_features=1024, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
  )
  (2): Sequential(
    (0): Linear(in_features=512, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
  )
)


In [17]:
sample_input = torch.rand((1, 3864)).to(device)
onnx_model_path = "asl_model.onnx"

model_infe.eval()

torch.onnx.export(
    model_infe,  # PyTorch Model
    sample_input,  # Input tensor
    onnx_model_path,  # Output file (eg. 'output_model.onnx')
    opset_version=12,  # Operator support version
    input_names=["input"],  # Input tensor name (arbitary)
    output_names=["output"],  # Output tensor name (arbitary)
    dynamic_axes={"input": {0: "input"}},
)

In [18]:
import onnx
from onnx_tf.backend import prepare

tf_model_path = "/kaggle/working/tf_model"
onnx_model = onnx.load(onnx_model_path)
tf_rep = prepare(onnx_model)
tf_rep.export_graph(tf_model_path)

# Final Inference Model in Tensorflow

In [19]:
import tensorflow as tf


class ASLInferModel(tf.Module):
    def __init__(self):
        super(ASLInferModel, self).__init__()
        self.feature_gen = FeatureGen()
        self.model = tf.saved_model.load(tf_model_path)
        self.feature_gen.trainable = False
        self.model.trainable = False

    @tf.function(
        input_signature=[
            tf.TensorSpec(shape=[None, 543, 2], dtype=tf.float32, name="inputs")
        ]
    )
    def call(self, input):
        output_tensors = {}
        features = self.feature_gen(tf.cast(input, dtype=tf.float32))
        output_tensors["outputs"] = self.model(**{"input": features})["output"][0, :]
        return output_tensors

mytfmodel = ASLInferModel()
tf.saved_model.save(
    mytfmodel,
    "/kaggle/working/tf_infer_model",
    signatures={"serving_default": mytfmodel.call},
)

# Submission

In [20]:
# Convert the model

tf_infer_model_path = "/kaggle/working/tf_infer_model"
converter = tf.lite.TFLiteConverter.from_saved_model(tf_infer_model_path)
tflite_model = converter.convert()

tflite_model_path = "model.tflite"

# Save the model
with open(tflite_model_path, "wb") as f:
    f.write(tflite_model)

In [21]:
ROWS_PER_FRAME = 543  # number of landmarks per frame
pq_path = "/kaggle/input/asl-signs/train_landmark_files/53618/1001379621.parquet"

import tflite_runtime.interpreter as tflite

interpreter = tflite.Interpreter(tflite_model_path)
interpreter.allocate_tensors()

found_signatures = list(interpreter.get_signature_list().keys())

# if REQUIRED_SIGNATURE not in found_signatures:
#     raise KernelEvalException('Required input signature not found.')

prediction_fn = interpreter.get_signature_runner("serving_default")
output = prediction_fn(inputs=load_relevant_data_subset(pq_path))
sign = np.argmax(output["outputs"])

print(sign, output["outputs"].shape)

30 (250,)


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [22]:
!zip submission.zip $tflite_model_path

  adding: model.tflite (deflated 7%)
